In [25]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")


In [12]:
from penzai import pz
import json

from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
import jax.numpy as jnp
import numpy as np
import random
from penzai.data_effects.side_output import SideOutputValue
from micrlhf.utils.activation_manipulation import add_vector


In [4]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2-2b-it.gguf",
                                         from_type="gemma2",
                                         load_eager=True
                                         )

RuntimeError: Unable to initialize backend 'tpu': ABORTED: The TPU is already in use by process with pid 1484045. Not attempting to load libtpu.so in this process. (set JAX_PLATFORMS='' to automatically choose an available backend)

In [ ]:
from transformers import AutoTokenizer
import jax
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [10]:
from sprint.task_vector_utils import load_tasks, ICLDataset, ICLSequence
tasks = load_tasks()

In [26]:
import json, numpy as np

with open("cleanup_results_gemma_2_algo.jsonl", "r") as f:
    cleanup_results = [json.loads(line) for line in f][18:]


# with open("cleanup_results_gemma_2_all.jsonl", "r") as f:
#     tmp = [json.loads(line) for line in f]
#     tmp = [x for x in tmp if not x["task"].startswith("algo")]

#     cleanup_results += tmp

In [27]:

layers = [14, 16, 18, 20, 22, 24]

In [28]:
from micrlhf.utils.load_sae import sae_encode, get_dm_res_sae

thresholds = {
    layer: get_dm_res_sae(layer, load_65k=True).get("threshold", 0) for layer in layers
}


In [35]:
task = "present_simple_past_simple"
task_results = [result for result in cleanup_results if result["layer"] in layers and result["task"] == task] 

print(len(task_results))

6


In [36]:
# with open("gemma_2_cleaning_compact_65k.jsonl", "w") as f:
for r in task_results:
    task = r["task"]
    layer = r["layer"]
    
    weights = np.array(r["weights"])

    i = np.argwhere(weights > thresholds[layer]).flatten()
    w = weights[i]

    idx = np.argsort(w)[::-1]

    i = i[idx]
    w = w[idx]

    data = {
        "task": task,
        "weights": w.tolist(),
        "indices": i.tolist(),
        "tv loss": r["tv_loss"],
        "cleaning loss": r["loss"],
        "ito loss": r["ito_loss"],
        "layer": layer
    }

    print(data)
    # f.write(json.dumps(data) + "\n")

{'task': 'present_simple_past_simple', 'weights': [99.33623504638672, 80.54329681396484, 58.63016128540039, 34.633750915527344, 31.384580612182617, 12.398971557617188, 12.3705415725708], 'indices': [7678, 57043, 17899, 5446, 18436, 5447, 55982], 'tv loss': 6.8125, 'cleaning loss': 2.34375, 'ito loss': 6.8125, 'layer': 14}
{'task': 'present_simple_past_simple', 'weights': [43.28398132324219, 42.81904220581055, 35.30847930908203, 34.42615509033203, 33.54819869995117, 29.884140014648438, 27.383426666259766, 18.86416244506836, 16.213579177856445, 7.187838554382324], 'indices': [34279, 61613, 39700, 29721, 43597, 27146, 2702, 61107, 47139, 37576], 'tv loss': 3.9375, 'cleaning loss': 0.734375, 'ito loss': 6.53125, 'layer': 16}
{'task': 'present_simple_past_simple', 'weights': [44.88591003417969, 40.624420166015625, 39.58631896972656, 37.01276397705078, 28.482345581054688, 27.461191177368164, 26.271484375, 23.297405242919922], 'indices': [47899, 46985, 63994, 57630, 45455, 35308, 36806, 23825